# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,9.69,62,95,11.99,SH,1696180823
1,1,port-aux-francais,-49.3500,70.2167,6.26,97,100,19.13,TF,1696180815
2,2,yellowknife,62.4560,-114.3525,8.29,88,75,6.17,CA,1696180549
3,3,ust-nera,64.5667,143.2000,-2.20,62,31,3.42,RU,1696180821
4,4,bosobolo,4.1833,19.9000,26.15,79,54,0.92,CD,1696180951


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
%%capture --no-display


# Configure the map plot
humidity = city_data_df["Humidity"]



map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo= True, 
    tiles="OSM",
    frame_width= 900,
    frame_height =700,
    size= "Humidity",
    color = "City",
    alpha=.75)

# Display the map
map_plot



### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = city_data_df.loc[(city_data_df["Max Temp"] > 15) & (city_data_df["Max Temp"] < 20)
                              & (city_data_df["Wind Speed"] < 2.5) & (city_data_df["Cloudiness"] > 15)].reset_index(drop=True)

narrowed_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,13,knjazevac,43.5648,22.2502,18.49,75,50,1.34,RS,1696180953
1,55,alpine,32.8350,-116.7664,17.05,85,75,1.79,US,1696180962
2,89,sarvar,47.2539,16.9352,16.92,52,44,0.68,HU,1696180969
3,114,portland,45.5234,-122.6762,15.43,78,20,2.24,US,1696181046
4,124,sparti,37.0733,22.4297,19.97,62,85,2.14,GR,1696180977


In [10]:
# Drop any rows with null values
narrowed_df.isnull().sum()

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       0
Date          0
dtype: int64

In [11]:
clean_df = narrowed_df.loc[:,~(narrowed_df.isnull().any())]
clean_df.reset_index(drop=True)
# Display sample data
clean_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,13,knjazevac,43.5648,22.2502,18.49,75,50,1.34,RS,1696180953
1,55,alpine,32.8350,-116.7664,17.05,85,75,1.79,US,1696180962
2,89,sarvar,47.2539,16.9352,16.92,52,44,0.68,HU,1696180969
3,114,portland,45.5234,-122.6762,15.43,78,20,2.24,US,1696181046
4,124,sparti,37.0733,22.4297,19.97,62,85,2.14,GR,1696180977


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,knjazevac,RS,43.5648,22.2502,75,
1,alpine,US,32.8350,-116.7664,85,
2,sarvar,HU,47.2539,16.9352,52,
3,portland,US,45.5234,-122.6762,78,
4,sparti,GR,37.0733,22.4297,62,
5,san marcos,US,33.1434,-117.1661,67,
6,pacific grove,US,36.6177,-121.9166,83,
7,bijie,CN,27.3125,105.2942,97,
8,waliso,ET,8.5333,37.9833,67,
9,malango,ID,-2.9627,119.9001,99,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = radius = 10000
lat = ""
lon = ""
categories= "accommodation.hotel"
filters = f"circle:{lon},{lat},{radius}"
bias = f"proximity:{lon},{lat}"
limit = 5

params = {
    "categories":categories,
    "filter": filters,
    "bias": bias,
    "limit": limit,
    "lang": "en",
    "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("STARTING HOTEL SEARCH")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionary
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    # print(name_address)
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df    

STARTING HOTEL SEARCH
knjazevac - nearest hotel: No hotel found
alpine - nearest hotel: No hotel found
sarvar - nearest hotel: No hotel found
portland - nearest hotel: No hotel found
sparti - nearest hotel: No hotel found
san marcos - nearest hotel: No hotel found
pacific grove - nearest hotel: No hotel found
bijie - nearest hotel: No hotel found
waliso - nearest hotel: No hotel found
malango - nearest hotel: No hotel found
la foa - nearest hotel: No hotel found
seaside - nearest hotel: No hotel found
mont-laurier - nearest hotel: No hotel found
oguz - nearest hotel: No hotel found
bunia - nearest hotel: No hotel found
torpoint - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,knjazevac,RS,43.5648,22.2502,75,No hotel found
1,alpine,US,32.8350,-116.7664,85,No hotel found
2,sarvar,HU,47.2539,16.9352,52,No hotel found
3,portland,US,45.5234,-122.6762,78,No hotel found
4,sparti,GR,37.0733,22.4297,62,No hotel found
5,san marcos,US,33.1434,-117.1661,67,No hotel found
6,pacific grove,US,36.6177,-121.9166,83,No hotel found
7,bijie,CN,27.3125,105.2942,97,No hotel found
8,waliso,ET,8.5333,37.9833,67,No hotel found
9,malango,ID,-2.9627,119.9001,99,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)